In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import os

PRESET = os.environ.setdefault("PRESET", "millionaires")
SELF_PARTY = os.environ.setdefault("SELF_PARTY", "sim")


In [ ]:
import ray
from opentelemetry import trace
from opentelemetry.sdk.environment_variables import OTEL_SERVICE_NAME

from secretnote.instrumentation import Profiler
from secretnote.instrumentation.sdk import (
    setup_tracing,
    setup_debug_exporter,
    inherit_tracing_context,
)
from secretnote.instrumentation.rules import create_default_rules

ray.shutdown()

create_default_rules()

os.environ[OTEL_SERVICE_NAME] = f"{PRESET}:{SELF_PARTY}"

setup_tracing()
# setup_debug_exporter()

with trace.get_tracer(__name__).start_as_current_span(
    "sim_trace",
    context=inherit_tracing_context(),
):
    with Profiler() as profiler:
        with open(f"presets/{PRESET}/_world.py") as f:
            _world = compile(f.read(), f.name, "exec")
        with open(f"presets/{PRESET}/_algorithm.py") as f:
            _algorithm = compile(f.read(), f.name, "exec")
        exec(_world, globals())
        exec(_algorithm, globals())


In [ ]:
from secretnote.display.app import visualize_run

visualize_run(profiler)
